In [3]:
import psycopg2
from psycopg2 import Error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [4]:
import pandas as pd
# Connection parameters, yours will be different
param_dic = {
    "host"      : "ec2-34-239-241-121.compute-1.amazonaws.com",
    "database"  : "dfm419pf436p66",
    "user"      : "eulpledavzofae",
    "password"  : "07be1edaf3f3f3fef6702e318a67f7bfb3dde0268efc076985820060aef10f96"
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [5]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    #list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # turn it into a pandas dataframe
    heart_dp = pd.DataFrame(tupples, columns=column_names)
    return heart_dp

In [7]:
# Connect to the database
conn = connect(param_dic)
column_names = ["Pt_ID", "Age", "Sex", "ChestPainType", "RestingBP", "Cholesterol", "FastingBS","RestingEKG","MaxHR","ExerciseAngina","OldPeak","ST_Slope", "HeartDisease"]
# Execute the "SELECT *" query
heart_dp_test = postgresql_to_dataframe(conn, "select * from heart_dp_test", column_names)
heart_dp_test.head()

Connecting to the PostgreSQL database...
Connection successful


,Pt_ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingEKG,MaxHR,ExerciseAngina,OldPeak,ST_Slope,HeartDisease
0,10157,43,M,ASY,110,211,0,Normal,161,N,0,Up,Absence
1,10436,45,M,ASY,115,260,0,LVH,185,N,0,Up,Absence
2,10437,64,M,ASY,145,212,0,LVH,132,N,2,flat,Presence
3,10474,52,M,ASY,108,233,1,Normal,147,N,0.1,Up,Absence
4,10477,48,M,NAP,124,255,1,Normal,175,N,0,Up,Absence


In [8]:
heart_dp_train = postgresql_to_dataframe(conn, "select * from heart_dp_train", column_names)
heart_dp_train.head()

,Pt_ID,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingEKG,MaxHR,ExerciseAngina,OldPeak,ST_Slope,HeartDisease
0,25047,74,M,NAP,138,0,0,Normal,116,N,0.2,Up,Absence
1,25177,60,M,ASY,130,186,1,ST,140,Y,0.5,Flat,Presence
2,25498,40,M,ASY,125,0,1,Normal,165,N,0,Flat,Presence
3,25189,69,M,ASY,140,208,0,ST,140,Y,2,Flat,Presence
4,25434,62,M,ASY,158,210,1,Normal,112,Y,3,Down,Presence
